## Forecasting Australian Dollar to Indian Rupee weekly exchange rates for the next 12 weeks using LSTM with HyperModel

## Libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import xlsxwriter
import plotly.express as px
from scipy.stats import skew, kurtosis
import tensorflow as tf
import keras_tuner as kt
from keras_tuner import HyperModel
import os
import datetime
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from pyts.image import GramianAngularField
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from keras import backend as K
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.layers import LeakyReLU

2024-11-21 12:24:43.235360: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 12:24:43.329524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-21 12:24:43.363822: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-21 12:24:43.374194: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 12:24:43.461509: I tensorflow/core/platform/cpu_feature_guar

## data

In [2]:
# Defining the start and end dates
start_date = '2005-01-01'
end_date = '2024-10-31'

# Downloading historical data of Australian Dollar to Indian Rupee from Yahoo Finance
data = yf.download('AUDINR=X', start=start_date, end=end_date)['Adj Close'].resample("W").mean().ffill()

data

[*********************100%***********************]  1 of 1 completed


Ticker,AUDINR=X
Date,
2005-01-09,33.406600
2005-01-16,33.239000
2005-01-23,33.121400
2005-01-30,33.709600
2005-02-06,33.603401
...,...
2024-10-06,57.701152
2024-10-13,56.615109
2024-10-20,56.304502


We use the previous 12 weeks rates to predict that of the upcoming 12 weeks.

In [3]:
window_size = 12
X = data.values
X_time = data.index
Xdf = []
ydf = []
time = []
for i in range(0, len(X)-2*window_size+1): 
    Xdf.append(X[i:i+window_size])
    ydf.append(X[i+window_size:i+2*window_size])
    time.append(X_time[i+window_size:i+2*window_size])

Xdf = np.array(Xdf, dtype='float32')
ydf = np.array(ydf, dtype='float32')

## split

In [4]:
ts = 200#int(0.4 * len(data)) 

test_time = time[-ts:]

len(test_time)

200

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    Xdf, ydf, test_size=ts, shuffle=False
)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(812, 12, 1) (200, 12, 1) (812, 12, 1) (200, 12, 1)


In [6]:
X_train_scaled = []
scaler_input = StandardScaler()
X_train_scaled.append(scaler_input.fit_transform(X_train[0]))
for i in range(1, len(X_train)):
    X_train_scaled.append(scaler_input.transform(X_train[i]))
X_train_scaled = np.array(X_train_scaled)


In [7]:
y_train_scaled = []
scaler_output = StandardScaler()
y_train_scaled.append(scaler_output.fit_transform(y_train[0]))
for i in range(1, len(y_train)):
    y_train_scaled.append(scaler_output.transform(y_train[i]))
y_train_scaled = np.array(y_train_scaled)


In [8]:
X_test_scaled = []
for i in range(0, len(X_test)):
    X_test_scaled.append(scaler_input.transform(X_test[i]))
X_test_scaled = np.array(X_test_scaled)


## model

In [9]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)
val_split = 0.2
SEED = 42

class LSTM_model(HyperModel):
    def build(self, hp):
        
        #act_fun = "LeakyReLU"
        # We do ask the Keras Tuner to choose whether is best to have a dropout rate after each hidden layer of 0.1 or 0.2 or 0.3
        n_dropout = hp.Choice("n_dropout", values=[0.10, 0.20, 0.30])

        model = tf.keras.models.Sequential()

        model.add(
                LSTM(
                    units=hp.Int(
                        "input_unit", min_value=12, max_value=36, step=12
                    ),
                    return_sequences=True,
                    #activation=act_fun,
                    input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]),
                )
        )
        model.add(LeakyReLU(alpha=0.1))

        # Now, we will use a loop to let the tuner choose the number of layers that is best for the model between 1 and 3
        for i in range(1, hp.Int("num_layers", 1, 3)):
            # Within this loop, we will also ask the tuner to decide the optimal number of units that each of the selected layer should have.
            model.add(
                LSTM(
                    units=hp.Int(
                        f'lstm_{i}_units', min_value=36, max_value=60, step=12
                    ),
                    return_sequences=True,
                    #activation=act_fun,
                    )
            )
            model.add(LeakyReLU(alpha=0.1))
            model.add(Dropout(n_dropout, seed=SEED))


        model.add(
                LSTM(
                    units=hp.Int(
                        f'lstm_units', min_value=12, max_value=36, step=12
                    ),
                    return_sequences=False,
                    #activation=act_fun,
                )
            )
        model.add(LeakyReLU(alpha=0.1))
        model.add(Dropout(n_dropout, seed=SEED))


        for i in range(1, hp.Int("num_layers", 1, 2)):
            # Within this loop, we will also ask the tuner to decide the optimal number of units that each of the selected layer should have.
            model.add(
                Dense(
                    units=hp.Int(
                        "units_dense_" + str(i), min_value=12, max_value=24, step=12
                    ),
                    #activation=act_fun,
                )
            )
            model.add(LeakyReLU(alpha=0.1))
            model.add(Dropout(n_dropout, seed=SEED))

        model.add(Dense(units=12))

        hp_lr = hp.Choice("hp_lr", values=[1e-5, 1e-3, 1e-1])
        adam = tf.keras.optimizers.Adam(learning_rate=hp_lr)
        model.compile(optimizer=adam, loss="mean_absolute_error")

        return model

In [10]:
# First, we clear the session just to make sure our seeds are correctly working and replicability is achieved
K.clear_session()

# Then, we call the model and perform the tuning:
hypermodel = LSTM_model()
tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective("val_loss", direction="min"),
    overwrite=True,
    max_epochs=100,
    seed=42,
    directory=os.path.expanduser("~/keras_tuner"), #os.path.normpath("C:/"),
)

# Let's run the tuner! (Warning: this could take time)
tuner.search(X_train_scaled,
            y_train_scaled, 
            validation_split=val_split)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 254 Complete [00h 00m 42s]
val_loss: 4.170945167541504

Best val_loss So Far: 2.8994696140289307
Total elapsed time: 00h 34m 08s


In [11]:
best_hps.values

{'n_dropout': 0.2,
 'input_unit': 24,
 'num_layers': 1,
 'lstm_units': 36,
 'hp_lr': 0.1,
 'lstm_1_units': 48,
 'units_dense_1': 12,
 'lstm_2_units': 36,
 'units_dense_2': 12,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 34,
 'tuner/bracket': 3,
 'tuner/round': 3,
 'tuner/trial_id': '0203'}

In [12]:
model = tuner.hypermodel.build(best_hps)

es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    mode="min",
    verbose=1,
    patience=20,
    restore_best_weights=True,
)

# fit the model
history = model.fit(
    X_train_scaled,
    y_train_scaled,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    verbose=2,
    callbacks=[es],
)

Epoch 1/100
21/21 - 1s - 71ms/step - loss: 26.6013 - val_loss: 24.1380
Epoch 2/100
21/21 - 0s - 12ms/step - loss: 13.4471 - val_loss: 4.9309
Epoch 3/100
21/21 - 0s - 11ms/step - loss: 9.5140 - val_loss: 5.0129
Epoch 4/100
21/21 - 0s - 13ms/step - loss: 7.8378 - val_loss: 4.3625
Epoch 5/100
21/21 - 0s - 13ms/step - loss: 7.8527 - val_loss: 8.8318
Epoch 6/100
21/21 - 0s - 12ms/step - loss: 6.5649 - val_loss: 4.1308
Epoch 7/100
21/21 - 0s - 13ms/step - loss: 6.2452 - val_loss: 3.3884
Epoch 8/100
21/21 - 0s - 11ms/step - loss: 6.3104 - val_loss: 3.3699
Epoch 9/100
21/21 - 0s - 12ms/step - loss: 6.0456 - val_loss: 3.7845
Epoch 10/100
21/21 - 0s - 11ms/step - loss: 6.2949 - val_loss: 4.1404
Epoch 11/100
21/21 - 0s - 12ms/step - loss: 6.4968 - val_loss: 5.3433
Epoch 12/100
21/21 - 0s - 11ms/step - loss: 6.3426 - val_loss: 3.0256
Epoch 13/100
21/21 - 0s - 11ms/step - loss: 5.9318 - val_loss: 4.8221
Epoch 14/100
21/21 - 0s - 13ms/step - loss: 5.7499 - val_loss: 4.4029
Epoch 15/100
21/21 - 0s - 

In [13]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 12, 24)         │         2,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 12, 24)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 36)             │         8,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 36)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 36)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 12)             │           444 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,174 (137.40 KB)

 Trainable params: 11,724 (45.80 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 23,450 (91.61 KB)

In [14]:
y_train_pred = model.predict(X_train_scaled)

y_train_pred_trans = []
for i in range(0,len(y_train_pred)):
    y_train_pred_trans.append(y_train_pred[i])

y_train_pred_tran = scaler_output.inverse_transform(y_train_pred_trans)

train_mape = mean_absolute_percentage_error(y_train_pred_tran, y_train.reshape(y_train_pred_tran.shape))
train_mape

26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


0.026674613672022198

### test

In [15]:
y_pred = model.predict(X_test_scaled)
y_test_pred = scaler_output.inverse_transform(y_pred)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [16]:
y_test = y_test.reshape(y_test_pred.shape)

mae = mean_absolute_error(y_test_pred, y_test)
print(f"Mean Absolute Error (MAE): {np.round(float(mae), 4)}")

mse = mean_squared_error(y_test_pred, y_test)
print(f"Mean Squared Error (MSE): {np.round(float(mse), 4)}")

rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {np.round(float(rmse), 4)}")

r2 = r2_score(y_test_pred, y_test)
print(f"R-squared (R²): {np.round(float(r2), 4)}")

mape = mean_absolute_percentage_error(y_test_pred, y_test)
print(f"Mean Absolute Percentage Error (MAPE): {np.round(float(mape), 4)}")

Mean Absolute Error (MAE): 1.2502
Mean Squared Error (MSE): 2.5612
Root Mean Squared Error (RMSE): 1.6004
R-squared (R²): -2.38
Mean Absolute Percentage Error (MAPE): 0.0232


In [17]:
initial_path = "/mnt/d/LSTM_HyperModel"

#tf.saved_model.save(model, f'{initial_path}/BTC_LSTM')

model.save(f'{initial_path}/LSTM.h5')

Result = pd.DataFrame()

MAPE = []

workbook = xlsxwriter.Workbook(f"{initial_path}/Results.xlsx")
workbook.close()

for i in range(len(test_time)):
    
    Result['Report Time'] = test_time[i]
    Result['Real_Value'] = y_test[i]
    Result['Forecast'] = y_test_pred[i]
    Result["MAPE"] = abs( (Result['Real_Value'] - Result['Forecast']) / Result['Real_Value'] ) * 100
    Result["Overall_MAPE"] = np.average(Result["MAPE"])
    
    MAPE.append(np.average(Result["MAPE"]))

    with pd.ExcelWriter(f"{initial_path}/Results.xlsx", mode="a", engine="openpyxl") as writer:
        Result.to_excel(writer, sheet_name=f'{i+1}', index=False)


In [20]:
def error_fig(mapes, mypath):
    mape_results = pd.DataFrame()
    mape_results['MAPE'] = mapes
    mape_results.index = mape_results.index + 1
    
    mean = np.round(float(np.mean(mape_results)), 2) 
    std = np.round(float(np.std(mape_results)), 2)

    fig = px.line(
        mape_results, 
        labels={"value": 'MAPE', "index": 'Period'}, 
        title=f"Overall Avg. MAPE={mean} with std deviation={std}", 
        markers=True
    )
    fig.write_image(f"{mypath}/Overall_Mean.png", width=1186, height=360, scale=2)
    fig.show()
    return mean, std, mape_results

mean, std, mape_results = error_fig(MAPE, initial_path)
mean, std

/home/akn/miniconda3/envs/tf216/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning:

The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)

/tmp/ipykernel_390124/2894061561.py:7: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



(2.25, 1.24)